In [1]:
#change working directory
import os
print(os.getcwd())
os.chdir('/home/student/A2/drive-download-20240507T073745Z-001')

/home/student/A2


In [3]:
"""
Part b
Task 1. Processing Data Stream (45%)
Question a
"""
from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient
#Residence wifi ip address: 172.16.62.212
ipadd = "10.192.33.112"

client = MongoClient(ipadd, 27017)
db = client.fit3182_db
collection = db.fit3182_assignment_db

#res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)
res = collection.find().sort('date', pymongo.DESCENDING).limit(1)

for d in res:
    res = d
#     print(d)

latest_date = res['date']
#string to datetime object
# latest_date = datetime.strptime(latest_date, '%Y-%m-%d %H:%M:%S')
#string to datetime object
latest_date = datetime.strptime(latest_date, '%d/%m/%Y')
start_date = latest_date + timedelta(days=1)
print(start_date)

2024-01-01 00:00:00


In [4]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import json
import pandas as pd

hostip = ipadd # change it to your IP

def connect_kafka_producer():
    producer = None
    
    try:
        producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],api_version=(0,10))
    
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
    finally:
        return producer 
    
def publish_msg(producer, topic, data):
    try:
        #print(data)        #send the  value by dumps to JSON object
                            #then encode with utf-8 into bytes
        value_bytes = json.dumps(data).encode('utf-8')
        producer.send(topic, value=value_bytes)
        #print(data)
        """
        The flush() method in Kafka’s producer API is used to ensure that all 
        previously sent messages have been transmitted to the server and acknowledged 
        before proceeding. When you call flush(), it blocks the current thread and waits 
        for the producer to complete the sending of all records
        """
        producer.flush()  
    except Exception as ex:
        print('Exception from publish_msg func')
        print(str(ex))
    


In [5]:
"""
Event Producer 1
"""
climate_stream_csv = pd.read_csv("./climate_streaming.csv", sep=',')
climate_stream_csv.head()

,latitude,longitude,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2
0,-37.623,149.323,19,56.8,7.9,11.1,0.00I,154
1,-38.038,142.986,15,50.7,9.2,13.0,0.02G,128
2,-37.950,142.366,16,53.6,8.1,15.0,0.00G,133
3,-38.231,147.172,24,61.6,7.7,14.0,0.00I,186
4,-37.903,145.250,24,62.3,7.0,13.0,0.00I,185


In [ ]:
"""
Main Function
"""

if __name__ == '__main__':
    topic = 'Producer1'
    print('Publishing records..')
    producer01 = connect_kafka_producer()


    # Assuming 'climate' is your DataFrame
    #print(len(climate.index))
    rand_row_indices = list(range(len(climate_stream_csv.index)))
    random.shuffle(rand_row_indices)  
    #print(len(indices))

    for index, climate_row in climate_stream_csv.iterrows():
        climate_row = climate_stream_csv.loc[rand_row_indices[index]]

        #inner-join
    #     if fires_list:
            #print(fires_list)

        #left-outer join
        #print(start_date.strftime('%Y-%m-%d'))
        climate_doc = {
                                        #string from time with format, 
                                        #since MongoDB can not store object
            #.strftime('%d/%m/%Y')
            #convert datetime object to string in format
            # laster dumps not work on datetime object but number of string
            'latitude': float(climate_row['latitude']),
            'longitude': float(climate_row['longitude']),
            'air_temperature_celcius': int(climate_row['air_temperature_celcius']),
            'relative_humidity': float(climate_row['relative_humidity']),
            'windspeed_knots': float(climate_row['windspeed_knots']),
            'max_wind_speed': float(climate_row['max_wind_speed']),
            #'precipitation ' blank space count as char in csv column
            # get to do try except, return None if column not exist
            #'precipitation ': str(climate_row.get(['precipitation '])),
            #'precipitation': str(climate_row.get(['precipitation'])),
            'precipitation': str(climate_row['precipitation']),
            'GHI_w/m2': int(climate_row['GHI_w/m2']),
            'producer': "Producer_1",
            'date': start_date.strftime('%Y-%m-%d'),
            'created_time': int(10)

        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        print(climate_doc)
        publish_msg(producer01, topic, climate_doc)
        start_date += timedelta(days=1)
        # change to 1 for debugging
        sleep(10)
        #sleep(10) # every 10 seconds send




Publishing records..
{'latitude': -37.63, 'longitude': 149.232, 'air_temperature_celcius': 18, 'relative_humidity': 57.0, 'windspeed_knots': 7.4, 'max_wind_speed': 15.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 145, 'producer': 'Producer_1', 'date': '2024-01-01', 'created_time': 10}
{'latitude': -37.583, 'longitude': 149.316, 'air_temperature_celcius': 25, 'relative_humidity': 58.3, 'windspeed_knots': 12.2, 'max_wind_speed': 22.0, 'precipitation': ' 0.04G', 'GHI_w/m2': 199, 'producer': 'Producer_1', 'date': '2024-01-02', 'created_time': 10}
{'latitude': -36.7801, 'longitude': 142.7151, 'air_temperature_celcius': 16, 'relative_humidity': 51.7, 'windspeed_knots': 7.4, 'max_wind_speed': 15.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 135, 'producer': 'Producer_1', 'date': '2024-01-03', 'created_time': 10}
{'latitude': -37.8088, 'longitude': 142.2291, 'air_temperature_celcius': 11, 'relative_humidity': 40.8, 'windspeed_knots': 6.4, 'max_wind_speed': 8.9, 'precipitation': ' 0.00G', 'GHI_w/m2': 1

{'latitude': -36.834, 'longitude': 142.524, 'air_temperature_celcius': 12, 'relative_humidity': 47.3, 'windspeed_knots': 9.7, 'max_wind_speed': 14.0, 'precipitation': ' 0.02G', 'GHI_w/m2': 105, 'producer': 'Producer_1', 'date': '2024-02-02', 'created_time': 10}
{'latitude': -37.38, 'longitude': 148.053, 'air_temperature_celcius': 14, 'relative_humidity': 51.0, 'windspeed_knots': 9.9, 'max_wind_speed': 15.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 119, 'producer': 'Producer_1', 'date': '2024-02-03', 'created_time': 10}
{'latitude': -37.5915, 'longitude': 143.0015, 'air_temperature_celcius': 17, 'relative_humidity': 52.0, 'windspeed_knots': 8.8, 'max_wind_speed': 15.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 143, 'producer': 'Producer_1', 'date': '2024-02-04', 'created_time': 10}
{'latitude': -36.404, 'longitude': 142.5467, 'air_temperature_celcius': 25, 'relative_humidity': 60.9, 'windspeed_knots': 6.3, 'max_wind_speed': 8.9, 'precipitation': ' 0.00I', 'GHI_w/m2': 195, 'producer': 'Produc

{'latitude': -37.485, 'longitude': 148.095, 'air_temperature_celcius': 11, 'relative_humidity': 45.4, 'windspeed_knots': 5.2, 'max_wind_speed': 8.9, 'precipitation': ' 0.00A', 'GHI_w/m2': 98, 'producer': 'Producer_1', 'date': '2024-03-05', 'created_time': 10}
{'latitude': -36.9817, 'longitude': 143.5051, 'air_temperature_celcius': 14, 'relative_humidity': 44.0, 'windspeed_knots': 14.4, 'max_wind_speed': 23.9, 'precipitation': ' 0.16G', 'GHI_w/m2': 126, 'producer': 'Producer_1', 'date': '2024-03-06', 'created_time': 10}
{'latitude': -35.954, 'longitude': 141.076, 'air_temperature_celcius': 10, 'relative_humidity': 42.1, 'windspeed_knots': 9.4, 'max_wind_speed': 15.0, 'precipitation': ' 0.13B', 'GHI_w/m2': 91, 'producer': 'Producer_1', 'date': '2024-03-07', 'created_time': 10}
{'latitude': -37.386, 'longitude': 148.043, 'air_temperature_celcius': 8, 'relative_humidity': 43.3, 'windspeed_knots': 3.9, 'max_wind_speed': 8.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 72, 'producer': 'Producer_1

{'latitude': -35.2881, 'longitude': 142.5679, 'air_temperature_celcius': 17, 'relative_humidity': 38.7, 'windspeed_knots': 16.8, 'max_wind_speed': 22.9, 'precipitation': ' 0.00I', 'GHI_w/m2': 159, 'producer': 'Producer_1', 'date': '2024-04-06', 'created_time': 10}
{'latitude': -37.391, 'longitude': 148.066, 'air_temperature_celcius': 14, 'relative_humidity': 44.6, 'windspeed_knots': 7.7, 'max_wind_speed': 15.9, 'precipitation': ' 0.03G', 'GHI_w/m2': 125, 'producer': 'Producer_1', 'date': '2024-04-07', 'created_time': 10}
{'latitude': -37.294, 'longitude': 141.232, 'air_temperature_celcius': 12, 'relative_humidity': 49.0, 'windspeed_knots': 4.6, 'max_wind_speed': 8.9, 'precipitation': ' 0.00G', 'GHI_w/m2': 104, 'producer': 'Producer_1', 'date': '2024-04-08', 'created_time': 10}
{'latitude': -37.8387, 'longitude': 144.6301, 'air_temperature_celcius': 11, 'relative_humidity': 41.4, 'windspeed_knots': 7.7, 'max_wind_speed': 15.0, 'precipitation': ' 0.03G', 'GHI_w/m2': 101, 'producer': 'Pro

In [ ]:
# """
# Main Function
# """
# if __name__ == '__main__':
#     topic = 'Scenaries01'
#     print('Publishing records..')
#     producer01 = connect_kafka_producer()


#     # Assuming 'climate' is your DataFrame
#     #print(len(climate.index))
#     rand_row_indices = list(range(len(climate.index)))
#     random.shuffle(rand_row_indices)  
#     #print(len(indices))

#     for index, climate_row in climate.iterrows():
#         climate_row = climate.loc[rand_row_indices[index]]

#         #inner-join
#     #     if fires_list:
#             #print(fires_list)

#         #left-outer join
#         #print(start_date.strftime('%Y-%m-%d'))
#         climate_doc = {
#             'stations': str(climate_row['station']),
#                                         #string from time with format, 
#                                         #since MongoDB can not store object
#             #.strftime('%d/%m/%Y')
#             #convert datetime object to string in format
#             # laster dumps not work on datetime object but number of string
#             'date': start_date.strftime('%Y-%m-%d'),
#             'air_temperature_celcius': int(climate_row['air_temperature_celcius']),
#             'relative_humidity': float(climate_row['relative_humidity']),
#             'windspeed_knots': float(climate_row['windspeed_knots']),
#             'max_wind_speed': float(climate_row['max_wind_speed']),
#             'precipitation': str(climate_row['precipitation']),
#             'GHI_w/m2': int(climate_row['GHI_w/m2']),
#             'producer': "Producer_1"

#         }
#         #update start date
#         #print(climate_doc['date'])
#         #producer publish msg in strictured streaming
#         # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
#         # msg = json.dumps(climate_doc)
#         print(climate_doc)
#         publish_msg(producer01, topic, climate_doc)
#         start_date += timedelta(days=1)
#         # change to 1 for debugging
#         sleep(1)
#         #sleep(10) # every 10 seconds send


